# Final Project 1: A Neural Algorithm of Artistic Style

## Dependencies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torchvision.transforms as transforms
import math
import copy
from tqdm import tqdm
from PIL import Image

In [2]:
# Mounting Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Defining Model

In [20]:
# Set the device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load and preprocess content and style images
def load_image(image_path, image_size=None):
    transform = transforms.Compose([
        transforms.Resize(image_size) if image_size is not None else transforms.Resize(512),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)
    return image

content_image = load_image("/content/drive/MyDrive/Projects/CS180FinalProj1/Data/campanile.jpg", image_size=(512, 512))
style_image = load_image("/content/drive/MyDrive/Projects/CS180FinalProj1/Data/basquiat.jpg", image_size=(512, 512))

# Display the content and style images
def imshow(tensor, title=None, show=False, save_fig=False, save_path=""):
    tensor = tensor.squeeze(0).detach().cpu()

    inv_normalize = transforms.Normalize(
        mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
        std=[1/0.229, 1/0.224, 1/0.225]
    )
    tensor = inv_normalize(tensor)

    tensor.clamp_(0, 1)

    image = transforms.ToPILImage()(tensor)

    plt.imshow(image)
    plt.axis("off")
    if title is not None:
        plt.title(title)
    if save_fig:
        plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
    if show:
        plt.show()


plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
imshow(content_image, title="Content Image")

plt.subplot(1, 2, 2)
imshow(style_image, title="Style Image")
plt.show()

# Define the VGG19 model with modified layers for feature extraction
class VGG19Features(nn.Module):
    def __init__(self, avg_pool: bool=False):
        super(VGG19Features, self).__init__()
        vgg19_layers = models.vgg19(pretrained=True).features
        print(vgg19_layers)

        if avg_pool:
            vgg19_layers = nn.Sequential(*[nn.AvgPool2d(2) if isinstance(x, nn.MaxPool2d) else x for x in vgg19_layers])

        self.slice1 = nn.Sequential()
        self.slice2 = nn.Sequential()
        self.slice3 = nn.Sequential()
        self.slice4 = nn.Sequential()
        self.slice5 = nn.Sequential()

        for x in range(1):
            self.slice1.add_module(str(x), vgg19_layers[x])
        for x in range(1, 6):
            self.slice2.add_module(str(x), vgg19_layers[x])
        for x in range(6, 11):
            self.slice3.add_module(str(x), vgg19_layers[x])
        for x in range(11, 20):
            self.slice4.add_module(str(x), vgg19_layers[x])
        for x in range(20, 29):
            self.slice5.add_module(str(x), vgg19_layers[x])

        for p in self.parameters():
            p.requires_grad = False

        self.eval()

    def forward(self, x):
        h1 = self.slice1(x)
        h2 = self.slice2(h1)
        h3 = self.slice3(h2)
        h4 = self.slice4(h3)
        h5 = self.slice5(h4)

        return [h1, h2, h3, h4, h5]

# Define content loss and style loss functions
def gram_matrix(input):
    n, c, h, w = input.shape
    x = input.view(n, c, h * w)
    y = input.view(n, c, h * w).permute(0, 2, 1)
    gram = torch.bmm(x, y) / (h * w)

    return gram

def calc_content_loss(input_features, target_features, weights):
    total_loss = torch.tensor(0.0, dtype=torch.float32).to(device)
    num_feats = len(input_features)
    for i in range(num_feats):
        block_loss = F.mse_loss(input_features[i], target_features[i])
        total_loss += block_loss * weights[i]

    return total_loss

def calc_style_loss(input_features, target_features, weights):
    total_loss = torch.tensor(0.0, dtype=torch.float32).to(device)
    num_feats = len(input_features)
    for i in range(num_feats):
        gram_input = gram_matrix(input_features[i])
        gram_target = gram_matrix(target_features[i])
        block_loss = F.mse_loss(gram_input, gram_target)
        total_loss += block_loss * weights[i]

    return total_loss

# Initialize the generated image with content image or random noise
input_image = content_image.clone().requires_grad_(True)

# Set the hyperparameters
style_weight = 1000000   # Weight for the style loss
content_weight = 1     # Weight for the content loss
lr = 0.1              # Learning rate
iterations = 2000       # Number of iterations
content_slice_weights = (0.0, 0.0, 0.0, 0.0, 1.0)    # Weights for each content layer
style_slice_weights = (0.2, 0.2, 0.2, 0.2, 0.2)      # Weights for each style layer

# Initializations
vgg = VGG19Features(avg_pool=True).to(device)
optimizer = optim.Adam([input_image], lr=lr)

with torch.no_grad():
    content_features = vgg(content_image)
    style_features = vgg(style_image)

# Training Loop
prog_bar = tqdm(range(1, iterations + 1))
for i in prog_bar:
    input_features = vgg(input_image)
    content_loss = calc_content_loss(input_features, content_features, content_slice_weights)
    style_loss = calc_style_loss(input_features, style_features, style_slice_weights)
    loss = content_weight * content_loss + style_weight * style_loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # with torch.no_grad():
        # input_image.clamp_(0, 1)
    if (i + 1) % 200 == 0 or i == 0:
        imshow(input_image, show=True, save_fig=True, save_path=f"/content/drive/MyDrive/Projects/CS180FinalProj1/Output/campanile_basquiat_{i}.jpg")
    prog_bar.set_description(f"Iteration [{i}/{iterations}] Loss: {loss.item():.4f}")


Output hidden; open in https://colab.research.google.com to view.